In [73]:
import numpy as np
from time import time

from numpy.testing import assert_allclose

# Part I. Construct a Householder reflection of a vector.

Given a vector $\mathbf{x}$, and a plane with a normal vector $\mathbf{u}$, the Householder transformation reflects $\mathbf{x}$ about the plane.

The matrix of the Householder transformation is

$$
\mathbf{H} = \mathbf{1} - 2 \mathbf{u} \mathbf{u}^T
$$

Given two equal-length vectors $\mathbf{x}$ and $\mathbf{y}$, a rotation which brings $\mathbf{x}$ to $\mathbf{y}$ is a Householder transform with

$$
\mathbf{u} = \frac{\mathbf{x} - \mathbf{y}}{\left|\mathbf{x} - \mathbf{y}\right|}
$$

Write a function which rotates a given vector, $\mathbf{x} = (x_1, \dots, x_n)$ into $\mathbf{y} = (\left|\mathbf{x}\right|, 0, \dots, 0)^T$ using a Householder transformation.

In [74]:
def householder(vec):
    """Construct a Householder reflection to zero out 2nd and further components of a vector.

    Parameters
    ----------
    vec : array-like of floats, shape (n,)
        Input vector
    
    Returns
    -------
    outvec : array of floats, shape (n,)
        Transformed vector, with ``outvec[1:]==0`` and ``|outvec| == |vec|``
    H : array of floats, shape (n, n)
        Orthogonal matrix of the Householder reflection
    """
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    #Creating y-vector
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec) 
    #Creating u-vector and H-matrix using formulas above
    u = (vec - y) / np.linalg.norm(vec - y)
    H = np.eye(u.shape[0]) - 2 * np.outer(u, u)
    
    return y, H

Test your function using tests below:

In [75]:
# Test I.1 (10% of the total grade).

v = np.array([1, 2, 3])
v1, h = householder(v)

assert_allclose(np.dot(h, v1), v)
assert_allclose(np.dot(h, v), v1)
#Assertion failed cause the tolerance is too low (x[2] ~ 0 but not a 'real' zero)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 33.3%
Max absolute difference: 0.
Max relative difference: nan
 x: array([ 3.741657,  0.      , -0.      ])
 y: array([3.741657, 0.      , 0.      ])

In [76]:
# Test I.2 (10% of the total grade).

rndm = np.random.RandomState(1234)

vec = rndm.uniform(size=7)
v1, h = householder(vec)

assert_allclose(np.dot(h, v1), vec)

# Part II. Compute the $\mathrm{QR}$ decomposition of a matrix.

Given a rectangular $m\times n$ matrix $\mathbf{A}$, construct a Householder reflector matrix $\mathbf{H}_1$ which transforms the first column of $\mathbf{A}$ (and call the result $\mathbf{A}^{(1)}$)

$$
\mathbf{H}_1 \mathbf{A} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
&& \dots&& \\
0      & \times & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(1)}\;.
$$

Now consider the lower-right submatrix of $\mathbf{A}^{(1)}$, and construct a Householder reflector which annihilates the second column of $\mathbf{A}$:

$$
\mathbf{H}_2 \mathbf{A}^{(1)} =%
\begin{pmatrix}
\times & \times & \times & \dots & \times \\
0      & \times & \times & \dots & \times \\
0      & 0      & \times & \dots & \times \\
&& \dots&& \\
0      & 0      & \times & \dots & \times \\
\end{pmatrix}%
\equiv \mathbf{A}^{(2)} \;.
$$

Repeating the process $n-1$ times, we obtain

$$
\mathbf{H}_{n-1} \cdots \mathbf{H}_2 \mathbf{H}_1 \mathbf{A} = \mathbf{R} \;,
$$

with $\mathbf{R}$ an upper triangular matrix. Since each $\mathbf{H}_k$ is orthogonal, so is their product. The inverse of an orthogonal matrix is orthogonal. Hence the process generates the $\mathrm{QR}$ decomposition of $\mathbf{A}$. 

Write a function, which receives a recangular matrix, $A$, and returns the Q and R factors of the $QR$ factorization of $A$.

In [97]:
def qr_decomp(a):
    """Compute the QR decomposition of a matrix.
    
    Parameters
    ----------
    a : ndarray, shape(m, n)
        The input matrix
    
    Returns
    -------
    q : ndarray, shape(m, m)
        The orthogonal matrix
    r : ndarray, shape(m, n)
        The upper triangular matrix
        
    Examples
    --------
    >>> a = np.random.random(size=(3, 5))
    >>> q, r = qr_decomp(a)
    >>> np.assert_allclose(np.dot(q, r), a)
    
    """
    R = np.array(a, copy=True, dtype=float)
    m, n = R.shape
    H = np.eye(m)
    
    for i in range(min(n, m)):
        #Creating a householder transformation for a column of R-matrix taking only the part below the diagonal
        h = householder(R[i:, i])[1]
        #Making a mxm H_i-matrix from (m-i)x(m-i) h-matrix, so it will be easier to apply the changes
        H_i = np.eye(m)
        H_i[i:, i:] = h
        #Forming two main matrixes
        R = H_i @ R
        H = H_i @ H
    #H.T = H.inv as it is orthogonal
    Q = H.T
    
    return Q, R

In [98]:
# Might want to turn this on for prettier printing: zeros instead of `1e-16` etc

np.set_printoptions(suppress=True)

In [131]:
# Test II.1 (20% of the total grade)

rndm = np.random.RandomState(1234)
a = rndm.uniform(size=(1000, 100))

start = time()

q, r = qr_decomp(a)

print('QR decomposition time: ', time() - start)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(q.shape[0]), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

QR decomposition time:  40.171870708465576


Now compare your decompositions to the library function (which actually wraps the corresponding LAPACK functions)

In [100]:
from scipy.linalg import qr
qq, rr = qr(a)

assert_allclose(np.dot(qq, rr), a)

assert_allclose(qq, q)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 55%
Max absolute difference: 1.26400256
Max relative difference: 2.
 x: array([[-0.064549, -0.265962,  0.054838, -0.287148, -0.203601, -0.024046,
         0.053599, -0.225802,  0.351513,  0.194273, -0.061765, -0.076939,
         0.397683,  0.068876, -0.24069 ,  0.058714,  0.16676 , -0.360956,...
 y: array([[ 0.064549,  0.265962,  0.054838,  0.287148,  0.203601,  0.024046,
        -0.053599,  0.225802,  0.351513,  0.194273, -0.061765,  0.076939,
         0.397683, -0.068876, -0.24069 ,  0.058714, -0.16676 , -0.360956,...

In [101]:
assert_allclose(rr, r)

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=0

Mismatch: 60.3%
Max absolute difference: 5.93411138
Max relative difference: 2.
 x: array([[-2.967056, -2.182505, -1.946538, ..., -1.555921, -1.716971,
        -1.659684],
       [ 0.      , -1.809394, -0.870327, ..., -1.220563, -0.554873,...
 y: array([[ 2.967056,  2.182505,  1.946538, ...,  1.555921,  1.716971,
         1.659684],
       [-0.      ,  1.809394,  0.870327, ...,  1.220563,  0.554873,...

Check if your `q` and `r` agree with `qq` and `rr`. Explain.

*Enter your explanation here* (10% of the total grade, peer-graded)
The differences are connected to the way how the matrixes are computed: QR decomposition has multiple pairs of Q and R that satisfie the equation A = QR where A is an initial matrix. But there will be only one pair if A-matrix is a square one, detA =/= 0 and there is a restriction for elements of R-matrix: they need to be positive. As you can see my method gives that type of matrix (with positive elements).

# Part III. Avoid forming Householder matrices explicitly.

Note the special structure of the Householder matrices: A reflector $\mathbf{H}$ is completely specified by a reflection vector $\mathbf{u}$. Also note that the computational cost of applying a reflector to a matrix strongly depends on the order of operations:

$$
\left( \mathbf{u} \mathbf{u}^T \right) \mathbf{A}  \qquad \textrm{is } O(m^2 n)\;,
$$
while
$$
\mathbf{u} \left( \mathbf{u}^T \mathbf{A} \right) \qquad \textrm{is } O(mn)
$$

Thus, it seems to make sense to *avoid* forming the $\mathbf{H}$ matrices. Instead, one stores the reflection vectors, $\mathbf{u}$, themselves, and provides a way of multiplying an arbitrary matrix by $\mathbf{Q}^T$, e.g., as a standalone function (or a class).

Write a function which constructs the `QR` decomposition of a matrix *without ever forming the* $\mathbf{H}$ matrices, and returns the $\mathbf{R}$ matrix and reflection vectors. 

Write a second function, which uses reflection vectors to multiply a matrix with $\mathbf{Q}^T$. Make sure to include enough comments for a marker to follow your implementation, and add tests. 

(Peer-graded, 40% of the total grade)

In [132]:
#The same method as above but without H-matrix, only u-vectors that are contrained in U-matrix
def uVector(vec):
    vec = np.asarray(vec, dtype=float)
    if vec.ndim != 1:
        raise ValueError("vec.ndim = %s, expected 1" % vec.ndim)
    #That's just a part of the previous code
    y = np.zeros_like(vec)
    y[0] = np.linalg.norm(vec)
    
    u = (vec - y) / np.linalg.norm(vec - y)
    
    return u

def ur_decomp(a):
    R = np.array(a, copy=True, dtype=float)
    m, n = R.shape
    U = np.zeros_like(a)
    
    for i in range(min(n, m)):
        u = uVector(R[i:, i])
        u_i = np.zeros(m)
        u_i[i:] = u
        U[:,i] = u_i
        #Using a modified method, firstly multiplying u_i and R
        R = R - 2 * np.outer(u_i, u_i @ R)
    
    return U, R

def u_to_q(u):
    U = np.array(u, copy=True, dtype=float)
    n, m = U.shape
    Q = np.eye(n)
    
    #Compiling Q-matrix using a standart formula and u-vectors
    for i in range(m):
        Q = Q - 2 * np.outer(U[:,i], U[:,i] @ Q)
        
    return Q.T

start = time()

u, r = ur_decomp(a)
q = u_to_q(u)

print('QR decomposition time: ', time() - start)

# test that Q is indeed orthogonal
assert_allclose(np.dot(q, q.T), np.eye(q.shape[0]), atol=1e-10)

# test the decomposition itself
assert_allclose(np.dot(q, r), a)

QR decomposition time:  14.570931911468506
